In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import heapq

In [3]:
test_news_path = '..infer/mind/large_test/news.tsv'
bias_prob_lstur_dir = '../infer/bias_prob_lstur/infer_large_lstur/'
bias_detection_path = 'data/0929_concat_mind_over_res.csv'
candidate_news_file = 'data/news_new_with_time.tsv'

In [4]:
with open(os.path.join(bias_prob_lstur_dir, 'news.pkl'), 'rb') as f:
    bias_prob_lstur_news = pickle.load(f)

In [5]:
type(bias_prob_lstur_news)

dict

In [6]:

for i in range(1, 6):
    with open(os.path.join(bias_prob_lstur_dir, f'user{i}.pkl'), 'rb') as f:
        bias_prob_lstur_user = pickle.load(f)

        v_list = []
        for k, v in bias_prob_lstur_user.items():
            v_list.append(v)

        v_vec = np.asarray(v_list)
        print(v_vec.shape)
        
        with open(os.path.join(bias_prob_lstur_dir, f'user{i}.npy'), 'wb') as f:
            pickle.dump(v_vec, f)

(2000, 400)
(2000, 400)
(2000, 400)
(2000, 400)
(2000, 400)


In [7]:
!wc -l $test_news_path

  120961 /Users/qin/2022/NewsRec/data/mind/large_test/news.tsv


In [8]:
news_df = pd.read_csv(test_news_path, sep='\t', header=None)
nid_list = news_df[0].tolist()

In [9]:
nid2vecs = dict()
for idx, nid in enumerate(nid_list):
    nid2vecs[nid] = bias_prob_lstur_news[idx+1]

In [10]:
news_df = pd.read_csv(candidate_news_file, sep='\t', header=None)
news_df.columns = ['nid', 'category', 'subcategory', 'title', 'abstract', 'url', 'entity1', 'entity2', 'timestamp']
news_df.shape

(22283, 9)

In [11]:
nid_list = news_df['nid'].tolist()

In [12]:
nid_vecs_arr = []
candidate_nid = []
for nid in nid_list:
    nid_vecs_arr.append(nid2vecs[nid])
    candidate_nid.append(nid)

In [13]:
with open(os.path.join(bias_prob_lstur_dir, f'candiate_nid_list.pkl'), 'wb') as f:
     pickle.dump(candidate_nid, f)

In [14]:
nid_vecs_arr = np.asarray(nid_vecs_arr)

In [15]:
nid_vecs_arr.shape

(22283, 400)

In [16]:
with open(os.path.join(bias_prob_lstur_dir, f'news.npy'), 'wb') as f:
     pickle.dump(nid_vecs_arr, f)

In [18]:

for j in range(1, 6):
    with open(os.path.join(bias_prob_lstur_dir, f'news.npy'), 'rb') as f:
         news_vec = pickle.load(f)
    with open(os.path.join(bias_prob_lstur_dir, f'user{j}.npy'), 'rb') as f:
         user_vec = pickle.load(f)
    with open(os.path.join(bias_prob_lstur_dir, f'candiate_nid_list.pkl'), 'rb') as f:
         candidate_nid = pickle.load(f)

    bias_detection_df = pd.read_csv(bias_detection_path, header=None, sep='\t')
    bias_detection_df.columns = ['nid', 'category', 'sub_category', 'bias_prob']

    nis2bias_prob = dict(zip(bias_detection_df['nid'], bias_detection_df['bias_prob']))

    a = np.dot(user_vec, news_vec.T)

    top_n = 100
    aa_arr = []
    for aa in a:
        ind = heapq.nlargest(top_n, range(len(aa)), aa.take)
        aa_prob_list = [nis2bias_prob[candidate_nid[idx]] for idx in ind]
        aa_arr.append(aa_prob_list)
    aa_arr = np.asarray(aa_arr)

    look_num_list = [i*10 for i in range(1, 11)]
    mean_val_list = []
    std_val_list = []
    display_score_list = []
    for look_num in look_num_list:
        bb_arr = aa_arr[:, :look_num]
        bias_prob_list = []
        for bb in bb_arr:
            temp = np.count_nonzero(bb >= 0.5)
            bias_prob_list.append(temp/len(bb))
        
        # computing the mean and std in float 64 are more accurate
        bias_prob_list = np.asarray(bias_prob_list)
        mean_val = np.mean(bias_prob_list, dtype=np.float64)
        std_val = np.std(bias_prob_list, dtype=np.float64)
        display_score = f'{mean_val*100:.2f}'
        mean_val_list.append(mean_val)
        std_val_list.append(std_val)
        display_score_list.append(display_score)
        
    print(f'group: {j}')
    print(f'mean: {mean_val_list}')
    print(f'std: {std_val_list}')
    print(f'display: {display_score_list}')
    print("*"*10)

group: 1
mean: [0.06484999999999999, 0.0671, 0.06756666666666666, 0.0680625, 0.06855000000000001, 0.068875, 0.06942142857142858, 0.0697625, 0.07015, 0.07063000000000001]
std: [0.08479078664571994, 0.05962457547018679, 0.052400922383739265, 0.04927913953946436, 0.04688173951550859, 0.046490571774166094, 0.04533044152191139, 0.044147195196863866, 0.04372080013217428, 0.04326202838517861]
display: ['6.48', '6.71', '6.76', '6.81', '6.86', '6.89', '6.94', '6.98', '7.02', '7.06']
**********
group: 2
mean: [0.08280000000000001, 0.08665, 0.08896666666666667, 0.088675, 0.08871000000000001, 0.08916666666666666, 0.08970714285714286, 0.0902125, 0.09025555555555555, 0.09059500000000001]
std: [0.10027043432637558, 0.07575801937748901, 0.06749846911844248, 0.06233874697970757, 0.060642690408655185, 0.05867163804693978, 0.057161389405572624, 0.05593231260863438, 0.055726261069979445, 0.05514885288924875]
display: ['8.28', '8.67', '8.90', '8.87', '8.87', '8.92', '8.97', '9.02', '9.03', '9.06']
********